In [7]:
import re
import os
import os
import collections
import fnmatch
import numpy as np
import pandas as pd
import pandas as pd
import scipy.io as sio
import shutil
from numpy import loadtxt

In [8]:
class ParsingError(Exception):
    pass

class P2mPaths:
    """Parse a p2m paths file"""

    # project.type.tx_y.rz.p2m
    _filename_match_re = (r'^(?P<project>.*)' +
                          r'\.' +
                          r'(?P<type>((doa)|(paths)|(cir)))' +
                          r'\.' +
                          r't(?P<transmitter>\d+)'+
                          r'_' +
                          r'(?P<transmitter_set>\d+)' +
                          r'\.' +
                          r'r(?P<receiver_set>\d+)' +
                          r'\.' +
                          r'p2m$')

    def __init__(self, filename):
        self.filename = filename
        self.file = None
        self._parse()

    def get_data_dict(self):
        return self.cvs_data
    
    def get_num_receiver(self):
        return self.n_receivers

    def _parse(self):
        with open(self.filename) as self.file:
            self._parse_meta()
            self._parse_header()
#             self.data = collections.OrderedDict()
            self.cvs_data = collections.OrderedDict()
            for rec in range(self.n_receivers):
                self._parse_receiver()

    def _parse_meta(self):
        match = re.match(P2mPaths._filename_match_re,
                         os.path.basename(self.filename))

        self.project = match.group('project')
        self.transmitter_set = int(match.group('transmitter_set'))
        self.transmitter = int(match.group('transmitter'))
        self.receiver_set = int(match.group('receiver_set'))

    def _parse_header(self):
        """read the first line of the file, indicating the number of receivers"""
        line = self._get_next_line()
        self.n_receivers = int(line.strip())

    def _get_next_line(self):
        """Get the next uncommedted line of the file

        Call this only if a new line is expected
        """
        if self.file is None:
            raise ParsingError('File is closed')
        while True:
            next_line = self.file.readline()
            if next_line == '':
                raise ParsingError('Unexpected end of file')
            if re.search(r'^\s*#', next_line, re.DOTALL):
                continue
            else:
                return next_line

    def _parse_receiver(self):
        """Get receiver and number of paths (pair Tx-Rx)"""
        line = self._get_next_line()
        receiver, n_paths = [int(i) for i in line.split()]
        self.cvs_data[receiver] = collections.OrderedDict()
        if n_paths == 0:

            self.cvs_data[receiver] = None
            return
        """Read: received_power -self.data[receiver][0]-, arrival_time -self.data[receiver][1]-,
         spread_delay -self.data[receiver][2]-"""
        #These are statistics per receiver (accounts for all paths)
        line = self._get_next_line()
        received_power, arrival_time, spread_delay = [float(i) for i in line.split()]

        
        self.cvs_data[receiver]['total_received_power'] = received_power
        self.cvs_data[receiver]['mean_arrival_time'] = arrival_time
        self.cvs_data[receiver]['spread_delay'] = spread_delay
        self.cvs_data[receiver]['paths_number'] = n_paths
        
        """Read for version 3.2: srcvdpower, arrival_time, arrival_angle1, arrival_angle2, departure_angle1, departure_angle2"""
        """or read for version 3.3: srcvdpower, phase, arrival_time, arrival_angle1, arrival_angle2, departure_angle1, departure_angle2"""
        #now get statistics per path
        for rays in range(0,n_paths):
            line = self._get_next_line()
            line_values_as_list = line.split() #split line and organize values as list
            if len(line_values_as_list) == 8: #version 3.2
                ray_n, n_interactions, srcvdpower, arrival_time, arrival_angle1, arrival_angle2, departure_angle1, departure_angle2 = [float(i) for i in line.split()]
            elif len(line_values_as_list) == 9: #version 3.3
                ray_n, n_interactions, srcvdpower, phase, arrival_time, arrival_angle1, arrival_angle2, departure_angle1, departure_angle2  = [float(i) for i in line.split()]
            else:
                raise Exception(line + ' has ' + len(line_values_as_list) + ' but was expecting 8 or 9!')
            interactions_list = self._get_next_line().strip()
            ray_n = int(ray_n)
            n_interactions = int(n_interactions)
            
            self.cvs_data[receiver][str(rays+1)+'_srcvdpower'] = srcvdpower
            
            if len(line_values_as_list) == 9: #version 3.3
            
                self.cvs_data[receiver][str(rays+1)+'_phase'] = phase
            self.cvs_data[receiver][str(rays+1)+'_arrival_time'] = arrival_time
            self.cvs_data[receiver][str(rays+1)+'_arrival_angle1'] = arrival_angle1
            self.cvs_data[receiver][str(rays+1)+'_arrival_angle2'] = arrival_angle2
            self.cvs_data[receiver][str(rays+1)+'_departure_angle1'] = departure_angle1
            self.cvs_data[receiver][str(rays+1)+'_departure_angle2'] = departure_angle2
            self.cvs_data[receiver][str(rays+1)+'_interactions_list'] = interactions_list
            self.cvs_data[receiver][str(rays+1)+'_interactions'] = collections.OrderedDict()
            self.cvs_data[receiver][str(rays+1)+'_n_interactions'] = n_interactions #store integer
            
            """Get coordinates of interactions"""
            for i in range(n_interactions+2): #add 2 to take in account Tx and Rx
                
                line = self._get_next_line()
                sp_line = line.split()
                interaction = i 
                coordinates = np.array([float(j) for j in sp_line[0:]])
              
                self.cvs_data[receiver][str(rays+1)+'_interactions'][str(interaction)] = coordinates
                if rays == 0 and i == 0 :
                    Tx_coordinates = coordinates
                    self.cvs_data[receiver]['Tx_coordinates'] = Tx_coordinates
                if rays == 0 and i == n_interactions+1:
                    Rx_coordinates = coordinates
                    self.cvs_data[receiver]['Rx_coordinates'] = Rx_coordinates

In [9]:
def cvsGen(projectDirectory, projectName):
    
    cvsPath = ['.\\cvsResults\\' + name for name in projectName]
    for i in range(len(cvsPath)):
        os.makedirs(cvsPath[i])
    
    for j in range(len(projectDirectory)):
        directory = projectDirectory[j]
        resultPath = cvsPath[j]
        for file in os.listdir(directory):
            if file.endswith(".p2m"):
                if fnmatch.fnmatch(file,'*paths*'):
                    print('Processing File: ',file)
                    path = os.path.join(directory,file)
                    p2mpath = P2mPaths(path)
                    pd.DataFrame(pd.DataFrame.from_dict(p2mpath.get_data_dict()).transpose()).to_csv(os.path.join(resultPath,p2mpath.project+"_Tx_"+str(p2mpath.transmitter_set)+".csv"))
            
        tx_loc = []
        # rx_loc = []
        prev_tx = []

        # Check files in the specified directory
        for file in os.listdir(directory):
        # Process only p2m files
            if file.endswith(".p2m"):

                # Use path files for the TX locations
                if fnmatch.fnmatch(file,'*paths*'):

                    # Open file
                    path = os.path.join(directory,file)
                    with open(path) as openfileobject:
                        for line in openfileobject:
                            # Look for a any scattering interaction
                            # Note this may not work if the TX is in outage with every single RX
                            if fnmatch.fnmatch(line,'Tx-*'):
                                # Collect and process the TX location 
                                # (First entry in the interaction list)
                                curr_line = openfileobject.readline()
                                arr_line = curr_line.split()

                                new_tx = [arr_line[0], arr_line[1], arr_line[2]]

                                # Append only if this is an unrecorded TX location
                                if new_tx != prev_tx:
                                    tx_loc.append([arr_line[0], arr_line[1], arr_line[2]])
                                    prev_tx = new_tx

                            else:
                                continue

                # Use txloss files for the RX locations                 
                # elif fnmatch.fnmatch(file,'*txloss.t001*'):

                #     # Open file
                #     path = os.path.join(directory,file)
                #     with open(path) as openfileobject:
                #         for line in openfileobject:
                #             # Ignore header
                #             if re.search(r'^\s*#', line, re.DOTALL):
                #                 continue

                #             # The first non-header line contains the RX coordinates
                #             arr_line = line.split()

                #             # Append the new RX location
                #             rx_loc.append([arr_line[1], arr_line[2], arr_line[3]])

        # Convert to data frame            
        df_tx = pd.DataFrame(data = tx_loc,columns = ['x','y','z'])   
        # df_rx = pd.DataFrame(data = rx_loc,columns = ['x','y','z'])

        print('Exporting Tx pos')

        # Export as CSV
        output_tx = os.path.join(resultPath,'tx_pos.csv')
        # output_rx = os.path.join(directory,'rx_pos.csv')

        df_tx.to_csv(output_tx)   
        # df_rx.to_csv(output_rx) 
        print('Done')
        

In [10]:
def reach_value_array(keywords, npath, lineIndex, csv_data):
    
    names = []
    for i in range(npath):
        item = str(i+1)+keywords
        names.append(item)
    
    data_object = csv_data.loc[lineIndex[0], names]
    nparr = np.array(data_object).T
    
    for ind in lineIndex[1:]:
        data_object = csv_data.loc[ind, names]
        nparr0 = np.array(data_object).T
        nparr = np.vstack((nparr, nparr0))
    
    nparr_doule = nparr.astype(np.float64)
    return nparr_doule

def extractData(file_name, project_name, result_path, npath, lineIndex):
    
    csv_data = pd.read_csv(file_name+'.csv')
    srcvdpower = reach_value_array('_srcvdpower',npath, lineIndex, csv_data)
    srcvdpower = reach_value_array('_srcvdpower',npath, lineIndex, csv_data)
    arrival_time = reach_value_array('_arrival_time',npath, lineIndex, csv_data)
    arrival_angle1 = reach_value_array('_arrival_angle1',npath, lineIndex, csv_data)
    arrival_angle2 = reach_value_array('_arrival_angle2',npath, lineIndex, csv_data)
    departure_angle1 = reach_value_array('_departure_angle1',npath, lineIndex, csv_data)
    departure_angle2 = reach_value_array('_departure_angle2',npath, lineIndex, csv_data)   

    sio.savemat(result_path+'_data.mat', {'gain': srcvdpower, 'dly': arrival_time, 'aoaEl': arrival_angle1, 
                                'aoaAz':arrival_angle2, 'aodEl': departure_angle1, 'aodAz':departure_angle2})
    
# generate the mat files from cvs
def matGen(projectName):
    matPath = ['.\\matResults\\' + name for name in projectName]
    for i in range(len(matPath)):
        if os.path.exists(matPath[i]):
            shutil.rmtree(matPath[i])
        os.makedirs(matPath[i])
    
    npath = 25
    for j in range(len(projectName)):
        
        pjName = projectName[j]
        print("Start: " + pjName)
        
        topdownmap = loadtxt('.\\topDownMap\\'+
                         pjName + '.glb_topdownmap.out', delimiter=",", unpack=False)
        nx = topdownmap.shape[0]
        ny = topdownmap.shape[1]
        csv_data = pd.read_csv('.\\cvsResults\\' + pjName + '\\' + pjName + '_Tx_1' +'.csv')
        ntx = csv_data.shape[0]
        # zoom in map 3 times
        xIndex = np.arange(1, nx, 3)
        yIndex = np.arange(1, ny, 3)
        zoomMap = topdownmap[xIndex,:]
        zoomMap = zoomMap[:,yIndex]
        roomIndex = []
        for i in range(160):
            for j in range(160):
                if zoomMap[i][j] == 1 or zoomMap[i][j] == 2:
                    roomIndex.append([i,j])
        # convert index to line
        roomIndex = np.array(roomIndex)
        lineIndex = (159-roomIndex[:,0])*160 + roomIndex[:,1] # nparray
        roomIndex_doule = roomIndex.astype(np.float64)
        sio.savemat('.\\matResults\\' + pjName + '\\'
                    + pjName +'_roomIndex.mat', {'roomIndex': roomIndex})
        
        for n in range(10):
            fileName = '.\\cvsResults\\' + pjName + '\\' + pjName + '_Tx_' + str(n+1)
            resultPath = '.\\matResults\\' + pjName + '\\' + pjName + '_Tx_' + str(n+1)
            extractData(fileName,pjName,resultPath,npath,lineIndex)
    print('Done') 

In [19]:
pjDirectory = []
pjName = []
# read the 'DraftData' folder info, extract project name
for root, dirs, files in os.walk(".\\DraftData", topdown=False):
    for name in dirs:
        pjDirectory.append(os.path.join(root, name))
        pjName.append(name)
        
if len(pjName) > 0:
    
    finishedPj = open('.\\cvsResults\\finished.txt','r+')
    lines = finishedPj.readlines()
    print(lines)
    existPjDir = []
    existPjName = []
    for line in lines:
        
        existPjName.append(line.strip())
        existPjDir.append(os.path.join('.\\DraftData', line.strip()))

    for k in range(len(existPjName)):
        if existPjName[k] in pjName:
            pjDirectory.remove(existPjDir[k])
            pjName.remove(existPjName[k])
    
    if len(pjName) > 0:
        for k in range(len(pjName)):
            finishedPj.write(pjName[k]+'\n')

        print("Working on: ") 
        print(*pjName, sep = ", ")
        # generate the cvs from p2m
        cvsGen(pjDirectory, pjName)
        # generate the mat from cvs
        matGen(pjName)
        print("Finished")

    else:
        print("No new project")
        
    finishedPj.close()
    
else:
    print("No project in \DraftData folder")

['Adrian\n', 'Bowlus\n', 'Brevort\n', 'Capistrano\n']
Working on: 
Bowlus2
Processing File:  Bowlus2.paths.t001_01.r011.p2m
Processing File:  Bowlus2.paths.t001_02.r011.p2m
Processing File:  Bowlus2.paths.t001_03.r011.p2m
Processing File:  Bowlus2.paths.t001_04.r011.p2m
Processing File:  Bowlus2.paths.t001_05.r011.p2m
Processing File:  Bowlus2.paths.t001_06.r011.p2m
Processing File:  Bowlus2.paths.t001_07.r011.p2m
Processing File:  Bowlus2.paths.t001_08.r011.p2m
Processing File:  Bowlus2.paths.t001_09.r011.p2m
Processing File:  Bowlus2.paths.t001_10.r011.p2m
Exporting Tx pos
Done
Start: Bowlus2
Done
Finished


In [16]:
def txlocGen(projectName):
    
    matPath = ['.\\matResults\\' + name for name in projectName]
    cvsPath = ['.\\cvsResults\\' + name for name in projectName]
    
    for i in range(len(projectName)):
        
        txcvsPath = cvsPath[i]+'\\tx_pos.csv'
        # open cvs file
        csv_data = pd.read_csv(txcvsPath)
        ntx = 10 # setting is 10 tx
        nparr = np.zeros([ntx,3])
        
        for k in range(ntx):
            # coordinate = ['x', 'y', 'z']
            nparr[k,0] = csv_data.loc[k, 'x']
            nparr[k,1] = np.abs(csv_data.loc[k, 'y'])
            nparr[k,2] = csv_data.loc[k, 'z']

        nparr_doule = nparr.astype(np.float64)
        # store mat
        sio.savemat(matPath[i]+'\\'+projectName[i]+'_txloc.mat', {'txloc': nparr_doule})
        

In [17]:
# gen tx.mat
pjDirectory = []
pjName = []
# read the 'DraftData' folder info, extract project name
for root, dirs, files in os.walk(".\\cvsResults", topdown=False):
    for name in dirs:
        pjDirectory.append(os.path.join(root, name))
        pjName.append(name)     
txlocGen(pjName)

In [7]:
pjName

['Adrian', 'Bowlus', 'Brevort', 'Capistrano']

In [8]:
pjDirectory

['.\\cvsResults\\Adrian',
 '.\\cvsResults\\Bowlus',
 '.\\cvsResults\\Brevort',
 '.\\cvsResults\\Capistrano']

In [12]:
cvsPath = ['.\\cvsResults\\' + name for name in pjName]
txcvsPath = cvsPath[0]+'\\tx_pos.csv'
# open cvs file
csv_data = pd.read_csv(txcvsPath)
ntx = 10 # setting is 10 tx
csv_data.loc[0, 'y']

-19.8